In [ ]:
import getpass

In [ ]:
if 0:
    #host = "ldap://cluster-test.amazonaws.com:389"
    host = "ldap://cluster-test.amazonaws.com"
    #host = "ldaps://cluster-test.amazonaws.com:636"
    #host = "ldaps://cluster-test.amazonaws.com"

    username = "cluster-test\Admin"
    DCs = "DC=cluster-test,DC=amazonaws,DC=com"
    OU = f"OU=cluster-test,{DCs}"
    OU_Users = "OU=Users,{OU}"
    CN_Admin = f"CN=Admin,{OU_Users}"

elif 1:
    #host = "ldap://cluster-test3.amazonaws.com:389"
    host = "ldap://cluster-test3.amazonaws.com"
    #host = "ldaps://cluster-test3.amazonaws.com:636"
    #host = "ldaps://cluster-test3.amazonaws.com"

    username = "cluster-test3\Admin"
    DCs = "DC=cluster-test3,DC=amazonaws,DC=com"
    OU = f"OU=cluster-test3,{DCs}"
    OU_Users = "OU=Users,{OU}"
    CN_Admin = f"CN=Admin,{OU_Users}"

admin_pw = getpass.getpass("AD Admin password")


#### Dump all

In [ ]:
!ldapsearch -x -H {host} -b {DCs} -s sub -D "{username}" -w "{admin_pw}" > dump.txt

#### List all users

In [ ]:
!ldapsearch -x -H {host} -b "CN=Users,{DCs}" -s sub -D "{username}" -w "{admin_pw}" cn


#### Get specific user

In [ ]:
!ldapsearch -x -H {host} -b "CN=Users,{DCs}" -s sub -D "{username}" -w "{admin_pw}" cn="Guest"

#### Add an organization

In [ ]:
%%file dev1_add.ldif

# Organization "dev1"
dn: OU=dev1,OU=cluster-test,DC=cluster-test,DC=amazonaws,DC=com
objectClass: top
objectClass: organizationalUnit
ou: dev1
description: This is Development team1


In [ ]:
!ldapadd -x -H {host} -D {admin_dn} -w "{admin_pw}" -f ./dev1_add.ldif

#### Modify the organization's description field

In [ ]:
%%file dev1_modify.ldif
dn: OU=dev1,OU=cluster-test,DC=cluster-test,DC=amazonaws,DC=com
changetype: modify
replace: description
description: This is Development team1 (updated)


In [ ]:
!ldapmodify -x -H {host} -D {admin_dn} -w "{admin_pw}" -f ./dev1_modify.ldif

#### Add users under the organization

In [ ]:
%%file users_add.ldif

# User "user1"
dn: CN=user1,OU=dev1,OU=cluster-test,DC=cluster-test,DC=amazonaws,DC=com
objectClass: top
objectClass: person
objectClass: organizationalPerson
objectClass: user
cn: user1
description: This is User1

# User "user2"
dn: CN=user2,OU=dev1,OU=cluster-test,DC=cluster-test,DC=amazonaws,DC=com
objectClass: top
objectClass: person
objectClass: organizationalPerson
objectClass: user
cn: user2
description: This is User2

# User "user3"
dn: CN=user3,OU=dev1,OU=cluster-test,DC=cluster-test,DC=amazonaws,DC=com
objectClass: top
objectClass: person
objectClass: organizationalPerson
objectClass: user
cn: user3
description: This is User3


In [ ]:
!ldapadd -x -H {host} -D {admin_dn} -w "{admin_pw}" -f ./users_add.ldif

#### List all users under the organization

In [ ]:
!ldapsearch -x -H {host} -b "OU=dev1,OU=cluster-test,DC=cluster-test,DC=amazonaws,DC=com" -s sub -D "{username}" -w "{admin_pw}" objectClass=organizationalPerson name

#### Delete the user

In [ ]:
%%file users_delete.ldif
CN=user1,OU=dev1,OU=cluster-test,DC=cluster-test,DC=amazonaws,DC=com
CN=user2,OU=dev1,OU=cluster-test,DC=cluster-test,DC=amazonaws,DC=com
CN=user3,OU=dev1,OU=cluster-test,DC=cluster-test,DC=amazonaws,DC=com

In [ ]:
!ldapdelete -v -x -H {host} -D {admin_dn} -w "{admin_pw}" -f ./users_delete.ldif

#### Delete the organization

In [ ]:
%%file dev1_delete.ldif
OU=dev1,OU=cluster-test,DC=cluster-test,DC=amazonaws,DC=com


In [ ]:
!ldapdelete -v -x -H {host} -D {admin_dn} -w "{admin_pw}" -f ./dev1_delete.ldif